# Overview

This notebook demonstrates how to scan for TF binding motifs. The base GRN will be generated by combining the ATAC-seq peaks and motif information.

### Notebook file
Notebook file is available on CellOracle's GitHub page.
https://github.com/morris-lab/CellOracle/blob/master/docs/notebooks/02_motif_scan/02_atac_peaks_to_TFinfo_with_celloracle_20200801.ipynb


# 0. Import libraries

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import seaborn as sns

import os, sys, shutil, importlib, glob
from tqdm.notebook import tqdm

In [11]:
from celloracle import motif_analysis as ma
from celloracle.utility import save_as_pickled_object

In [12]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.rcParams['figure.figsize'] = (15,7)
plt.rcParams["savefig.dpi"] = 600

# 1. Rerefence genome data preparation
## 1.1. Check reference genome installation

Before starting the analysis, we need to make sure the reference genome data is installed. If not, please install the correct reference genome using the instructions below.

In [18]:
# PLEASE make sure reference genome is correct.
ref_genome = "mm10"

genome_installation = ma.is_genome_installed(ref_genome=ref_genome)
print(ref_genome, "installation: ", genome_installation)

mm10 installation:  True


## 1.2. Install reference genome (if refgenome is not installed)

In [19]:
if not genome_installation:
    import genomepy
    genomepy.install_genome(ref_genome, "UCSC")
else:
    print(ref_genome, "is installed.")

mm10 is installed.


# 2. Load data


## 2.0. Input data format

In this notebook, we explain how to make a base GRN.

Please look at the previous steps for details on preprocessing data for a base GRN

https://morris-lab.github.io/CellOracle.documentation/tutorials/base_grn.html#step1-scatac-seq-analysis-with-cicero



The scATAC-seq file needs to be converted in a csv file three columns:
- The first column is index.
- The second column is peak_id.
- The third column is gene_short_name.


A correctly formatted file will look like this:



<img src="https://raw.githubusercontent.com/morris-lab/CellOracle/master/docs/demo_data/processed_peak_csv.png">


We will load the .csv file as a `pandas.DataFrame` using pd.read_csv().


<img src="https://raw.githubusercontent.com/morris-lab/CellOracle/master/docs/demo_data/processed_peak_df.png">




## 2.0. Download demo data

You can download the demo file by running the following command.

Note: If the file download fails, please manually download and unzip the data.

https://raw.githubusercontent.com/morris-lab/CellOracle/master/docs/demo_data/processed_peak_file.csv


In [7]:
# Download file. 
!wget https://raw.githubusercontent.com/morris-lab/CellOracle/master/docs/demo_data/processed_peak_file.csv
    
# If you are using macOS, please try the following command.
#!curl -O https://raw.githubusercontent.com/morris-lab/CellOracle/master/docs/demo_data/processed_peak_file.csv


--2021-07-07 21:49:27--  https://raw.githubusercontent.com/morris-lab/CellOracle/master/docs/demo_data/processed_peak_file.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 569448 (556K) [text/plain]
Saving to: ‘processed_peak_file.csv’

processed_peak_file 100%[===================>] 556.10K  --.-KB/s    in 0.02s   

2021-07-07 21:49:27 (29.2 MB/s) - ‘processed_peak_file.csv’ saved [569448/569448]



## 2.1. Load processed peak data 

In [13]:
# Load annotated peak data.
peaks = pd.read_csv("processed_peak_file.csv", index_col=0)
peaks.head()

,peak_id,gene_short_name
0,chr10_100015291_100017830,Kitl
1,chr10_100486534_100488209,Tmtc3
2,chr10_100588641_100589556,4930430F08Rik
3,chr10_100741247_100742505,Gm35722
4,chr10_101681379_101682124,Mgat4c


## 2.1. Check data

In [16]:
# Define quality check fuction
def decompose_chrstr(peak_str):
    """
    Args:
        peak_str (str): peak_str. e.g. 'chr1_3094484_3095479'
        
    Returns:
        tuple: chromosome name, start position, end position
    """
    
    *chr_, start, end = peak_str.split("_")
    chr_ = "_".join(chr_)
    return chr_, start, end

from genomepy import Genome

def check_peak_format(peaks_df, ref_genome):
    """
    Check peak format. 
     (1) Check chromosome name. 
     (2) Check peak size (length) and remove sort DNA sequences (<5bp)
    
    """
    
    df = peaks_df.copy()
    
    n_peaks_before = df.shape[0]
    
    # Decompose peaks and make df
    decomposed = [decompose_chrstr(peak_str) for peak_str in df["peak_id"]]
    df_decomposed = pd.DataFrame(np.array(decomposed))
    df_decomposed.columns = ["chr", "start", "end"]
    df_decomposed["start"] = df_decomposed["start"].astype(np.int)
    df_decomposed["end"] = df_decomposed["end"].astype(np.int)
    
    # Load genome data
    genome_data = Genome(ref_genome)
    all_chr_list = list(genome_data.keys())
    
    
    # DNA length check
    lengths = np.abs(df_decomposed["end"] - df_decomposed["start"])
    
    
    # Filter peaks with invalid chromosome name
    n_threshold = 5
    df = df[(lengths >= n_threshold) & df_decomposed.chr.isin(all_chr_list)]
    
    # DNA length check
    lengths = np.abs(df_decomposed["end"] - df_decomposed["start"])
    
    # Data counting
    n_invalid_length = len(lengths[lengths < n_threshold])
    n_peaks_invalid_chr = n_peaks_before - df_decomposed.chr.isin(all_chr_list).sum()
    n_peaks_after = df.shape[0]
    
    #
    print("Peaks before filtering: ", n_peaks_before)
    print("Peaks with invalid chr_name: ", n_peaks_invalid_chr)
    print("Peaks with invalid length: ", n_invalid_length)
    print("Peaks after filtering: ", n_peaks_after)
    
    return df

In [20]:
peaks = check_peak_format(peaks, ref_genome)

Peaks before filtering:  15779
Peaks with invalid chr_name:  0
Peaks with invalid length:  2
Peaks after filtering:  15777


## 2.2. [Optional step] Load custom motifs

You can chose to use either a custom TF binding reference or CellOracle’s default motifs during the motif analysis. If you would like to use our default motifs, you can continue to the next step without loading any additional data.


If you would like to use a custom motif dataset, please choose one of the following options.

- Motifs provided by gimmemotifs
 >Gimmemotifs is a python package for motif analysis. It provides many motif dataset. https://gimmemotifs.readthedocs.io/en/stable/overview.html#motif-databases
 > 
 > Please use this notebook to learn how to load motif data from gimmemotifs database. 
 > https://github.com/morris-lab/CellOracle/blob/master/docs/notebooks/02_motif_scan/motif_data_preparation/01_How_to_load_gimmemotifs_motif_data.ipynb

- Custom motifs provided by CellOracle.
 
 >CellOracle also provides many motif datasets generated from CisBP. http://cisbp.ccbr.utoronto.ca/
 >
 >Please look at this notebook to learn how to load the CisBP motifs.https://github.com/morris-lab/CellOracle/blob/master/docs/notebooks/02_motif_scan/motif_data_preparation/02_How_to_load_CisBPv2_motif_data.ipynb


- Make your own custom motif data.
 >You can create custom motif data by yourself.
 >
 >Please look at this notebook to learn how to create your custom motif dataset.https://github.com/morris-lab/CellOracle/blob/master/docs/notebooks/02_motif_scan/motif_data_preparation/03_How_to_make_custom_motif.ipynb


# 3. Instantiate TFinfo object and search for TF binding motifs
The motif analysis module has a custom class, `TFinfo`. 
The TFinfo objectexecutes the steps below.

- Converts a peak data into a DNA sequences.
- Scans the DNA sequences searching for TF binding motifs.
- Post-processes the motif scan results.
- Converts data into appropriate format. You can convert data into base-GRN. The base GRN data can be formatted as either a python dictionary or pandas dataframe. This output will be the final base GRN used in the GRN model construction step.

## 3.1. Instantiate TFinfo object

In [16]:
# Instantiate TFinfo object
tfi = ma.TFinfo(peak_data_frame=peaks, 
                ref_genome=ref_genome) 

## 3.2. Motif scan


You can specify the TF binding motif data as follows. 

`tfi.scan(motifs=motifs)`

If you do not specify the motifs or set motifs to `None`, the default motifs will be loaded automatically.

- For mouse and human, "gimme.vertebrate.v5.0." will be used as the default motifs. 

- For another species, the species-specific TF binding motif data extracted from CisBP ver2.0 will be used.



In [ ]:
%%time
# Scan motifs. !!CAUTION!! This step may take several hours if you have many peaks!
tfi.scan(fpr=0.02, 
         motifs=None,  # If you enter None, default motifs will be loaded.
         verbose=True)

# Save tfinfo object
tfi.to_hdf5(file_path="test1.celloracle.tfinfo")

In [16]:
# Check motif scan results
tfi.scanned_df.head()

,seqname,motif_id,factors_direct,factors_indirect,score,pos,strand
0,chr10_100015291_100017830,GM.5.0.Homeodomain.0001,TGIF1,"ENSG00000234254, TGIF1",10.311002,1003,1
1,chr10_100015291_100017830,GM.5.0.Mixed.0001,,"SRF, EGR1",7.925873,481,1
2,chr10_100015291_100017830,GM.5.0.Mixed.0001,,"SRF, EGR1",7.321375,911,-1
3,chr10_100015291_100017830,GM.5.0.Mixed.0001,,"SRF, EGR1",7.276585,811,-1
4,chr10_100015291_100017830,GM.5.0.Nuclear_receptor.0002,NR2C2,"NR2C2, Nr2c2",9.067331,449,-1


We have the score for each sequence and motif_id pair.
In the next step we will filter the motifs with low scores.

# 4. Filtering motifs

In [15]:
# Reset filtering 
tfi.reset_filtering()

# Do filtering
tfi.filter_motifs_by_score(threshold=10)

# Format post-filtering results.
tfi.make_TFinfo_dataframe_and_dictionary(verbose=True)



# 5. Get final base GRN

## 5.1. Get results as a dataframe

In [17]:
df = tfi.to_dataframe()
df.head()

,peak_id,gene_short_name,9430076c15rik,Ac002126.6,Ac012531.1,Ac226150.2,Afp,Ahr,Ahrr,Aire,...,Znf784,Znf8,Znf816,Znf85,Zscan10,Zscan16,Zscan22,Zscan26,Zscan31,Zscan4
0,chr10_100015291_100017830,Kitl,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,chr10_100486534_100488209,Tmtc3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,chr10_100588641_100589556,4930430F08Rik,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,chr10_100741247_100742505,Gm35722,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,chr10_101681379_101682124,Mgat4c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# 6. Save results
We will use this information when constructing the GRN models later. Save the results.

In [19]:
# Save result as a dataframe
df = tfi.to_dataframe()
df.to_parquet(os.path.join(folder, "base_GRN_dataframe.parquet"))

# Or save the result as a dictionary.
#td = tfi.to_dictionary(dictionary_type="targetgene2TFs")
#save_as_pickled_object(td, os.path.join(folder, "TFinfo_targetgene2TFs.pickled"))

**We will use this base GRN data in the GRN construction section.**

https://morris-lab.github.io/CellOracle.documentation/tutorials/networkanalysis.html